In [11]:
import os
import requests
from typing import List, TypedDict

import dotenv


from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langgraph.graph import StateGraph, END

import weaviate
from weaviate.embedded import EmbeddedOptions
from langchain_community.vectorstores import Weaviate
import warnings
import logging

from typing import TypedDict, List


dotenv.load_dotenv()
# --- 1. Data Preparation ---

url = 'https://raw.githubusercontent.com/SajeelKA/AgenticSamples/refs/heads/main/resource.txt'
res = requests.get(url)

with open("resource1.txt", "w") as f:
    f.write(res.text)

loader = TextLoader("./resource1.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

# --- 2. Setup Weaviate (Embedded) ---

client = weaviate.Client(
    embedded_options=EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    client=client,
    by_text=False,
)

retriever = vectorstore.as_retriever()

# --- 3. Initialize LLM ---

llm = ChatOpenAI(
    model="gpt-4o-mini",  # modern replacement for gpt-3.5-turbo
    temperature=0
)

class RAGGraphState(TypedDict):
    question: str
    documents: List[Document]
    generation: str

# --- 5. Nodes ---

def retrieve_documents_node(state: RAGGraphState) -> RAGGraphState:
    question = state["question"]
    documents = retriever.invoke(question)
    return {
        "question": question,
        "documents": documents,
        "generation": ""
    }

def generate_response_node(state: RAGGraphState) -> RAGGraphState:
    question = state["question"]
    documents = state["documents"]

    template = """You are an assistant for question-answering tasks.
                Use the following pieces of retrieved context to answer the question.
                If you don't know the answer, say that you don't know.
                Use three sentences maximum and keep the answer concise.
                
                Question: {question}
                Context: {context}
                Answer:
                """

    prompt = ChatPromptTemplate.from_template(template)

    context = "\n\n".join([doc.page_content for doc in documents])

    rag_chain = prompt | llm | StrOutputParser()

    generation = rag_chain.invoke({
        "question": question,
        "context": context
    })

    return {
        "question": question,
        "documents": documents,
        "generation": generation
    }

# --- 6. Build Graph ---

workflow = StateGraph(RAGGraphState)

workflow.add_node("retrieve", retrieve_documents_node)
workflow.add_node("generate", generate_response_node)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

app = workflow.compile()

# --- 7. Run App ---

if __name__ == "__main__":
    print("\n--- Running RAG Query ---")
    query = "What happened to Jack?"
    inputs = {"question": query}

    for s in app.stream(inputs):
        if 'generate' in s.keys():
            print(s['generate']['generation'])

    print("\n--- Running another RAG Query ---")
    query_2 = "What happened to Jill?"
    inputs_2 = {"question": query_2}

    for s in app.stream(inputs_2):
        # print(s)
        if 'generate' in s.keys():
            print(s['generate']['generation'])



embedded weaviate is already listening on port 8079


{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"langchain_395b1489de9e42f8ad53861adc6ff137_e1aniIhoxP7F","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2026-02-17T14:02:14-05:00","took":142865}
{"action":"restapi_management","level":"info","msg":"Shutting down... ","time":"2026-02-17T14:02:14-05:00"}
{"action":"restapi_management","level":"info","msg":"Stopped serving weaviate at http://127.0.0.1:8079","time":"2026-02-17T14:02:14-05:00"}



--- Running RAG Query ---
Embedded weaviate wasn't listening on port 8079, so starting embedded weaviate again
Started /home/skhalid/.cache/weaviate-embedded: process ID 604375


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2026-02-17T14:02:15-05:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2026-02-17T14:02:15-05:00"}
{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"langchain_0de99c329b794413afc2bd7fa2079745_0nSUlC86ztMb","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2026-02-17T14:02:15-05:00","took":267661}
{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"langchain_16d2307bad634d97ba624b458ad97114_wsMsfoMRSllA","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2026-02-17T14:02:15-05:00","took":271205}
{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"langchain_321d55a7b6814147b1941b6c598d9530_

Jack fell down and broke his crown while going up the hill with Jill. After the fall, he got up and went home to rest and mend his head. He used vinegar and brown paper for his injury.

--- Running another RAG Query ---
Jill tumbled down the hill after Jack fell. The context does not specify any further details about her condition. Therefore, it is unclear what specifically happened to her.


In [ ]:
# !pip uninstall weaviate weaviate-client -y
# !pip install weaviate-client==3.25.3